In [6]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [7]:
import textwrap

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory


from rag_pipeline import split_text_add_video_metadata, clean_classification_text
from video_processing import analyze_video

from chat_memory import get_chat_history, last_n_messages

#### Jeff Nippard: Overhead Press transcript

In [8]:
jeff_ohp = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_nippard_ohp_dict"
jeff_ohp_chunked = split_text_add_video_metadata(jeff_ohp)
print("Chunks:", len(jeff_ohp_chunked))
print("Sample chunk:", jeff_ohp_chunked[:1])

Chunks: 11
Sample chunk: [Document(metadata={'video_id': '_RlRDWO2jfg', 'title': 'Build Bigger Shoulders With Perfect Training Technique (The Overhead Press)', 'author': 'Jeff Nippard', 'difficulty': 'intermediate', 'exercise_type': 'overhead_press'}, page_content="Okay, welcome everyone to a new episode of Technique Tuesday. This week, we're going to be looking at how to perform the overhead barbell press, or OHP, with perfect technique. With this movement, we're performing shoulder flexion, basically lifting your arm up overhead, which will be handled by the anterior or front deltoid and, to a lesser degree, the clavicular or upper head of the pecs. We'll also be performing elbow extension, which will hit all three heads of the triceps, and when viewed from the back, you can see that there will be scapular upward rotation occurring, handled by the upper traps. I like the overhead press for two main reasons. First, being a basic multi-joint barbell movement, it allows for a good deal 

#### Jeff Nippard: Bench Press transcript

In [9]:
jeff_bench = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_nippard_bench_dict"
jeff_bench_chunked = split_text_add_video_metadata(jeff_bench)
print("Chunks:", len(jeff_bench_chunked))
print("Sample chunk:", jeff_bench_chunked[:1])

Chunks: 16
Sample chunk: [Document(metadata={'video_id': 'vcBig73ojpE', 'title': 'How To Get A Huge Bench Press with Perfect Technique', 'author': 'Jeff Nippard', 'difficulty': 'intermediate', 'exercise_type': 'bench_press'}, page_content="Welcome, everyone, to the first episode of Technique Tuesday, where every week we're going to take an in-depth look at the lost art and science of training technique. Just as a quick general outline, for the most part, we will break each exercise into four sections. We will look at the muscles we'll be targeting, how to set up for the exercise, the execution of the movement, and common errors that I see many people make. So without further ado, let's jump right into it with the bench press exercise.")]


#### Jeff Nippard: Squat transcript

In [10]:
jeff_squat = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_nippard_squat_dict"
jeff_squat_chunked = split_text_add_video_metadata(jeff_squat)
print("Chunks:", len(jeff_squat_chunked))
print("Sample chunk:", jeff_squat_chunked[:1])

Chunks: 19
Sample chunk: [Document(metadata={'video_id': 'bEv6CCg2BC8', 'title': 'How To Get A Huge Squat With Perfect Technique (Fix Mistakes)', 'author': 'Jeff Nippard', 'difficulty': 'intermediate', 'exercise_type': 'squat'}, page_content="Okay, welcome everyone to a new episode of Technique Tuesday. This week, we're going to be looking at how to perform the squat with perfect technique. The back squat is often referred to as the king of lower body exercises and even the king of all exercises, and that's an appointment I think it actually deserves. Before we jump into the technique for this exercise, let's take a look at what muscles we're going to be targeting with this movement first.")]



#### Learning to Bench Press | The Starting Strength Method transcript

In [11]:
strength_bench = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_starting_strength_bench_dict"
strength_bench_chunked = split_text_add_video_metadata(strength_bench)
print("Chunks:", len(strength_bench_chunked))
print("Sample chunk:", strength_bench_chunked[:1])

Chunks: 6
Sample chunk: [Document(metadata={'video_id': 'rxD321l2svE', 'title': 'Learning to Bench Press | The Starting Strength Method', 'author': 'Mark Rippetoe', 'difficulty': 'advanced', 'exercise_type': 'bench_Press'}, page_content="When you're learning how to bench press, it might be prudent to use a spotter if one is available. However, if you're working inside a correctly set up power rack, a spotter is not absolutely necessary. Start with an empty bar. Lie down on the bench with your eyes looking straight up. In this position, you should be far enough down from the bar—meaning toward the foot end of the bench—that when you look up, your eyes are focused on the downside of the bar. Your feet should be flat on the ground with your shins approximately vertical. Your upper back should be flat against the bench with your lower back in an anatomically normal arched position. Take an overhand grip on the bar; your grip should be somewhere between 22 and 24 inches measured between the

#### How To Bench Press: Layne Norton's Complete Guide transcript



In [12]:
bodybuilding_bench = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_bodybuilding_bench_dict"
bodybuilding_bench_chunked = split_text_add_video_metadata(bodybuilding_bench)
print("Chunks:", len(bodybuilding_bench_chunked))
print("Sample chunk:", bodybuilding_bench_chunked[:1])

Chunks: 18
Sample chunk: [Document(metadata={'video_id': 'esQi683XR44', 'title': "How To Bench Press: Layne Norton's Complete Guide", 'author': 'Layne Norton', 'difficulty': 'beginner', 'exercise_type': 'bench_press'}, page_content="Research is my passion. Muscle and strength are my pursuits. I'm the powerlifter, bodybuilder, scientist, and coach. I'm Layne Norton. I am a physique architect. The bench press is one of the most important upper body exercises. This makes it crucial for overall development and strength. A lot of people think about it as just a chest movement, but it incorporates the pectorals, the triceps, the shoulders, and even the back when done correctly. When it's done wrong, the results can be disastrous. Seven years ago, I tore my right pectoral bench pressing incorrectly. But over time, I learned to incorporate more muscle groups and focus on the movement itself. Not only did it become more safe, but my results were better. In this video, I'm going to teach you thi

#### How To: Incline Barbell Bench Press | 3 GOLDEN RULES

In [13]:
incline_bench = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/processed/cleaned_incline_bench_dict.json"
incline_bench_chunked = split_text_add_video_metadata(incline_bench)
print("Chunks:", len(incline_bench_chunked))
print("Sample chunk:", incline_bench_chunked[:1])

Chunks: 6
Sample chunk: [Document(metadata={'video_id': 'SrqOu55lrYU', 'title': 'How To: Incline Barbell Bench Press | 3 GOLDEN RULES! (MADE BETTER!)', 'author': 'ScottHermanFitnesss', 'difficulty': 'beginner', 'exercise_type': 'incline_bench_press'}, page_content='What’s going on, Nation? I’m Scott from MuscularStrength.com, and today we’re going to go over the three golden rules for performing a barbell incline bench press. But before we get started, if you’ve been enjoying my Golden Rule series, be sure to smash that like and subscribe button, and let me know which exercise you want to see next down in the comment section below. Alright, guys, golden rule number one: you always want to make sure you’re benching with a grip just outside shoulder width. I know some of you are going to say that with a wider grip you can bench more weight, but when you have a wider grip on this exercise using a barbell, you’re not going to get much of a squeeze at the top of the movement. It should go w

#### Embed chunks as vectors into ChromaDB using OpenAI embedding model

In [14]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
#  embeddings tells Chroma how to turn your text into vectors. 
# You're passing it in so Chroma can embed each document's page_content before storing it.

all_docs = jeff_ohp_chunked + jeff_bench_chunked + jeff_squat_chunked + strength_bench_chunked + bodybuilding_bench_chunked + incline_bench_chunked


vectorstore = Chroma.from_documents(all_docs, embeddings, persist_directory="/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/chroma_db")
# index the documents and embedddings with a persistent directory in chroma db

print(vectorstore._collection.count())  # Should show your doc count now

152


# User video input

In [25]:
user_video = "/Users/chandlershortlidge/Desktop/Ironhack/fitness-form-coach/data/raw_workout_videos/user_01_bench.MP4"
user_video_encoded = analyze_video(filepath_in=user_video, frame_count=15, max_seconds=10)

len(user_video_encoded)

Frames processed: 15, (10s cap)
Saved to processed-images/user_01_bench
Video name: user_01_bench


15

In [26]:
encoded_images = []
for images in user_video_encoded:
    encoded_images.append({"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{images}"}})

# Router chain begins: gpt-4o

In [27]:

classification_image = user_video_encoded[0]

router_llm = ChatOpenAI(model='gpt-4o')

response = router_llm.invoke([

{"role": "user", "content": 
 
[{"type": "text", "text":  """Your job is to analyze images of users working out for proper form, and list the key checkpoints of their to body evaluate. 
  Give me ONLY the bodypart checkpoints. Do NOT include evaluation suggestions. Do NOT include an intro sentence. 
  Output format should be exactly the example below.
**Example**
  Overhead press

  1. Feet & base
  2. Glutes & legs
  3. Core & Ribcage
  4. Shoulder position
  5. Bar path
  6. Head & Neck
  7. Lockout position
  8. Tempo and control
   """},


    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{classification_image}"}}
    ]}


])

print(response.text)

**Bench Press**

1. Feet & base
2. Glutes & legs
3. Back & arch
4. Shoulder position
5. Grip & wrist alignment
6. Bar path
7. Head & Neck
8. Tempo and control


In [28]:
cleaned = clean_classification_text(response)
print(cleaned)

  Bench Press       Feet   base    Glutes   legs    Back   arch    Shoulder position    Grip   wrist alignment    Bar path    Head   Neck    Tempo and control


# Router query

In [19]:




# prompt = ChatPromptTemplate.from_messages([
#     ("system", 
     
#      """You are a world-class fitness coach. You have extensive experience in helping weight lifters achieve perfect form an maximum hypertrophy. 
# Your job is to analyze images of users lifting weights, offer them advice from your context, and to answer any questions they might have. 
# Inspect each image CLOSELY and Carefuly for problems or issues related to best practices in exercise form. Help the user diagnose their incorrect form. 
# Be specific about what you observe.

# # ANSWER CONTEXT
# Use ONLY the following context when answering a user: 
     
# ---   
# {context}
#  ---
# """
     
#      ),
#     MessagesPlaceholder(variable_name="history"),
#     ("human", "{query}")
# ])

# llm = ChatOpenAI(model='gpt-4o')

# output_parser = StrOutputParser()

# chain = prompt | llm | output_parser

# chain_with_history = RunnableWithMessageHistory(
#     chain,
#     get_session_history=get_chat_history,
#     input_messages_key="query",
#     history_messages_key="history"

# )

# response = chain_with_history.invoke(
#     {"query": "what's good grip for bench press?"},
#     config={"configurable": {"session_id": "test_123"}}
# )

# print(textwrap.fill(response))

In [37]:
user_query = "my elbow hurts when I do this"

# Image analysis LLM

In [115]:
retrieval_query = cleaned

results_user = vectorstore.similarity_search(user_query, k=5)
results_video = vectorstore.similarity_search(retrieval_query, k=5)
results = results_user + results_video

unique_results = []
seen = set()

for r in results:
    content = r.page_content
    if content not in seen:
        seen.add(content)
        unique_results.append(r)


context = "\n".join([r.page_content for r in unique_results])

llm = ChatOpenAI(model='gpt-5')

response = llm.invoke([

    {"role": "system", "content": 
     
F"""You are a world-class fitness coach. You have extensive experience in helping weight lifters achieve perfect form an maximum hypertrophy. 
Your job is to analyze images of users lifting weights, offer them advice from your contex›t, and to answer any questions they might have. 
Inspect each image CLOSELY and carefuly for problems or issues related to best practices in exercise form. Help the user diagnose their incorrect form. 
Be specific about what you observe.

# ANSWER CONTEXT
Use ONLY the following context when answering a user: 
     
---   
{context}
 ---

If the query or image isn't in context, reply, 'I don't have expert coaching content for this exercise yet. Currently I can analyze: bench press, overhead press, incline bench press...'"
"""
     
      },

      {"role": "user", "content": encoded_images},
      {"role": "user", "content": user_query}
    
    
    ]

)

print(response.text)

for i, doc in enumerate(unique_results, 1):
    print(f"==Results {i}==")
    print(textwrap.fill(doc.page_content, width=80))
    print(doc.metadata["exercise_type"])
    print("\n")

Thanks for the clips—this is a bench press, so I can help.

What I’m seeing that likely explains the elbow pain
- You’re pressing the bar “up” out of the J‑hooks. When the hooks are high, you have to use your shoulders to lift the bar before you can bring it out, which unpacks your shoulder blades. That usually makes the elbows take a beating on the first few reps.
- Your upper arms look flared as you start the rep, and at points the elbows aren’t stacked under the wrists. Flaring dumps tension into the shoulders and elbows and costs you power.
- Your feet come up on the toes—make sure they stay planted so you can stay tight through the whole set.

Fixes to try on your next session
- Lower the J‑hooks a notch or two and unrack out, not up. If you unrack by yourself, keep your butt elevated for the liftoff, then drop your hips once the bar is out over your start point. A spotter for the liftoff helps near heavier efforts.
- Golden rule #1: use a just‑outside‑shoulder‑width grip. Keep th

In [116]:
print(len(results_user))
print(len(results_video))
print(len(results))
print(len(unique_results))

5
5
10
6


In [114]:
for r in results:
    print(r.metadata)

{'difficulty': 'beginner', 'author': 'Layne Norton', 'exercise_type': 'bench_press', 'video_id': 'esQi683XR44', 'title': "How To Bench Press: Layne Norton's Complete Guide"}
{'title': "How To Bench Press: Layne Norton's Complete Guide", 'exercise_type': 'bench_press', 'author': 'Layne Norton', 'video_id': 'esQi683XR44', 'difficulty': 'beginner'}
{'difficulty': 'beginner', 'exercise_type': 'incline_bench_press', 'video_id': 'SrqOu55lrYU', 'title': 'How To: Incline Barbell Bench Press | 3 GOLDEN RULES! (MADE BETTER!)', 'author': 'ScottHermanFitnesss'}
{'exercise_type': 'incline_bench_press', 'video_id': 'SrqOu55lrYU', 'difficulty': 'beginner', 'title': 'How To: Incline Barbell Bench Press | 3 GOLDEN RULES! (MADE BETTER!)', 'author': 'ScottHermanFitnesss'}
{'difficulty': 'beginner', 'video_id': 'SrqOu55lrYU', 'exercise_type': 'incline_bench_press', 'title': 'How To: Incline Barbell Bench Press | 3 GOLDEN RULES! (MADE BETTER!)', 'author': 'ScottHermanFitnesss'}
{'difficulty': 'intermediate

In [113]:
# vectorstore cosine search: use if you need to verify GPT results
unique_results = vectorstore.similarity_search(retrieval_query, k=10)

for i, doc in enumerate(unique_results, 1):
    print(f"==Results {i}==")
    print(textwrap.fill(doc.page_content, width=80))
    print(doc.metadata["exercise_type"])
    print("\n")

==Results 1==
We've got the bench press setup down; now it's time to execute the set. We'll
break the execution into four phases: unrack, brace, descend, and press. First,
unrack the bar with your spotter's help, lifting out, not up, or, if unracking
by yourself, keep your butt elevated for the liftoff component, then drop your
hips once you've unracked. I prefer having a spotter near a maximal effort to
help with the liftoff. Ensure you have four main points of contact: your head,
upper back, glutes, and feet should all be planted. In position, take a deep
breath into your gut, pressing the air out against your belt if you have one.
You can cue yourself to puff your chest out to expand the rib cage as much as
possible. Grip the bar as hard as you can. You can optionally use the cue to
bend the bar or rip it in half, activating your upper back muscles to maintain
tightness. During the descent, drop your elbows at a 45-degree angle relative to
your torso when viewed from the top. There'

In [40]:
from chat_memory import get_chat_history, last_n_messages

In [ ]:
# response = chain_with_history.invoke(
#     {"query": "what about for overhead press?", "context": ""},
#     config={"configurable": {"session_id": "test_123"}}
# )

# print(textwrap.fill(response))

NameError: name 'chain_with_history' is not defined

In [ ]:
# for message in last_n_messages("test_123", 2):
#     print(message.type)
#     print(textwrap.fill(message.content))

human
what about for overhead press?
ai
Great question. For the overhead press, a “good” grip keeps the bar
stacked over your forearms, lets you clear your face smoothly, and
finishes with everything stacked overhead. Here’s how to set it:  -
Width: Just outside shoulder width. A solid start is pinkies 1–3
finger-widths inside the knurl rings. At the bottom (bar on upper
chest/clavicles), your forearms should be vertical from the front
view. - Hand placement: Bar sits low in the heel of your palm (not in
the fingers). Wrap the thumb. Keep wrists neutral to slightly extended
so the bar is directly over your forearm; knuckles angled slightly up.
- Elbow position: Slightly in front of the bar at the start (not
flared straight out and not behind the bar). This puts your forearm
perpendicular to the bar and sets a clean press path. - Bar path and
finish: Move your chin back to clear the bar, then “head through” as
it passes your forehead. Lock out with the bar over mid-foot, biceps
by ears,

In [ ]:
# # 3. A function that stores/retrieves chat history per session:

# chat_map = {}

# def get_chat_history(session_id):
#     if session_id not in chat_map:
#         chat_map[session_id] = InMemoryChatMessageHistory()
#     return chat_map[session_id]


# # 4. Wrap it all together:
# chain_with_history = RunnableWithMessageHistory(
#     chain,
#     get_session_history=get_chat_history,
#     input_messages_key="query",
#     history_messages_key="history"
)


In [105]:
# vectorstore cosine search: use if you need to verify GPT results
results = vectorstore.similarity_search(retrieval_query, k=5)

for i, doc in enumerate(results, 1):
    print(f"==Results {i}==")
    print(textwrap.fill(doc.page_content, width=80))
    print(doc.metadata["exercise_type"])
    print("\n")


==Results 1==
We've got the bench press setup down; now it's time to execute the set. We'll
break the execution into four phases: unrack, brace, descend, and press. First,
unrack the bar with your spotter's help, lifting out, not up, or, if unracking
by yourself, keep your butt elevated for the liftoff component, then drop your
hips once you've unracked. I prefer having a spotter near a maximal effort to
help with the liftoff. Ensure you have four main points of contact: your head,
upper back, glutes, and feet should all be planted. In position, take a deep
breath into your gut, pressing the air out against your belt if you have one.
You can cue yourself to puff your chest out to expand the rib cage as much as
possible. Grip the bar as hard as you can. You can optionally use the cue to
bend the bar or rip it in half, activating your upper back muscles to maintain
tightness. During the descent, drop your elbows at a 45-degree angle relative to
your torso when viewed from the top. There'

In [51]:
number_list = [3, 5, 5, 7, 7, 8, 8, 8, 8, 9, 11]

number_set = set(number_list)
num_list = list(number_set)
print(num_list)

[3, 5, 7, 8, 9, 11]


In [103]:

unique_results = []
seen = set()

results = results_user + results_video

for r in results:
    content = r.page_content
    if content not in seen:
        seen.add(content)
        unique_results.append(r)
print(unique_results)


    

[Document(id='50bcbba6-19c3-47c4-9186-da35e848f61a', metadata={'difficulty': 'beginner', 'author': 'Layne Norton', 'exercise_type': 'bench_press', 'video_id': 'esQi683XR44', 'title': "How To Bench Press: Layne Norton's Complete Guide"}, page_content="Another one of the most common mistakes I see is people flaring their elbows. Many people flare their elbows out because they believe this will lead to more chest recruitment. And even if it does, it's still not worth the danger. This was how I tore my pec seven years ago, by using too wide of a grip and flaring my elbows out. It also means you're not going to be able to keep your lats tight, and it's going to lead to less efficiency in the movement."), Document(id='a769e520-bf23-48bf-a990-e6b435f7b30f', metadata={'title': 'How To: Incline Barbell Bench Press | 3 GOLDEN RULES! (MADE BETTER!)', 'exercise_type': 'incline_bench_press', 'video_id': 'SrqOu55lrYU', 'author': 'ScottHermanFitnesss', 'difficulty': 'beginner'}, page_content='my elbo